In [11]:
from pathlib import Path
from src.config import PROJECT_ROOT

# Drei verschiedene Checkpoints – aktuell alle gleich,
# du kannst sie später durch echte ersetzen
CHECKPOINTS = [
    PROJECT_ROOT / "checkpoints" / "best_epoch1_val2.0773.pt",
    PROJECT_ROOT / "checkpoints" / "best_epoch3_val2.4214.pt",
    PROJECT_ROOT / "checkpoints" / "best_epoch2_val1.1781.pt",
]

# Quick check
for cp in CHECKPOINTS:
    print(cp, "exists:", cp.exists())



/home/daniel/PycharmProjects/mini_transformer/checkpoints/best_epoch1_val2.0773.pt exists: True
/home/daniel/PycharmProjects/mini_transformer/checkpoints/best_epoch3_val2.4214.pt exists: True
/home/daniel/PycharmProjects/mini_transformer/checkpoints/best_epoch2_val1.1781.pt exists: True


In [9]:
prompt1 = "alice was beginning to get very tired "

run_generation(
    cfg=cfg,
    checkpoint_path=CHECKPOINT_PATH,
    prompt=prompt1,
    max_new_tokens=200,
    temperature=0.8,
    max_vocab_size=256,
)


[device] Verwende Device für Inferenz: cuda
[gen] Lade Checkpoint: /home/daniel/PycharmProjects/mini_transformer/checkpoints/best_epoch3_val2.4214.pt
[gen] Vokabulargröße: 50
[checkpoint] Geladen von: /home/daniel/PycharmProjects/mini_transformer/checkpoints/best_epoch3_val2.4214.pt
[gen] Checkpoint-Epoche: 3, best_val_loss: 2.4213852882385254
[gen] Prompt: 'alice was beginning to get very tired '
[gen] Input-IDs-Länge: 40
===========================[ GENERIERTER TEXT ]===========================
che are in t aso,oy cote t ,iny s rou ,o,ed t sewad at there tin themsoge won the c sinory the can ty th af bo t thei
ondlfooud ,


In [6]:
import os
from pathlib import Path

print("CWD:", os.getcwd())
print("Inhalt hier:", os.listdir("."))


CWD: /home/daniel/PycharmProjects/mini_transformer/notebooks
Inhalt hier: ['01_inference_and_attention.ipynb']


In [15]:
def run_generation_for_checkpoint(prompt: str, checkpoint_path: Path):
    print("\n" + "="*60)
    print(f"CHECKPOINT: {checkpoint_path.name}")
    print("="*60)

    run_generation(
        cfg=cfg,
        checkpoint_path=str(checkpoint_path),
        prompt=prompt,
        max_new_tokens=200,
        temperature=0.8,
        max_vocab_size=256,
    )


In [17]:
prompt = "who is frankenstein"

for cp in CHECKPOINTS:
    run_generation_for_checkpoint(prompt, cp)



CHECKPOINT: best_epoch1_val2.0773.pt
[device] Verwende Device für Inferenz: cuda
[gen] Lade Checkpoint: /home/daniel/PycharmProjects/mini_transformer/checkpoints/best_epoch1_val2.0773.pt
[gen] Vokabulargröße: 50
[checkpoint] Geladen von: /home/daniel/PycharmProjects/mini_transformer/checkpoints/best_epoch1_val2.0773.pt
[gen] Checkpoint-Epoche: 1, best_val_loss: 2.077255763834523
[gen] Prompt: 'who is frankenstein'
[gen] Input-IDs-Länge: 21
===========================[ GENERIERTER TEXT ]===========================
e go
. turtle smkerf soon to proje
t wutenberw ele
troni
vmmthef lessons; said the xueenvmmhoc 
ould she cere little wrfphony ali

CHECKPOINT: best_epoch3_val2.4214.pt
[device] Verwende Device für Inferenz: cuda
[gen] Lade Checkpoint: /home/daniel/PycharmProjects/mini_transformer/checkpoints/best_epoch3_val2.4214.pt
[gen] Vokabulargröße: 50
[checkpoint] Geladen von: /home/daniel/PycharmProjects/mini_transformer/checkpoints/best_epoch3_val2.4214.pt
[gen] Checkpoint-Epoche: 3, 